In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 38.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
import pandas as pd

from collections import OrderedDict
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.5.0


In [ ]:
# Upload dataset
data = files.upload()

Saving merged_files_spectral.csv to merged_files_spectral.csv


In [ ]:
df_1 = pd.read_csv('Residential_2_kmeans.csv')
df_2 = pd.read_csv('Residential_4_kmeans.csv')


data_frames = [df_1, df_2]


columns_to_convert = ['hour']

for data_frame in data_frames:
    data_frame[columns_to_convert].applymap(lambda x: np.float32(x))
    # Replacing NaN energy values with the mean
    mean_energy = data_frame['energy_kWh'].mean()
    data_frame['energy_kWh'].fillna(value=mean_energy, inplace=True)

data_frames

[             date  hour  energy_kWh  kmeans_3  kmeans_4  kmeans_5  kmeans_6
 0      2016-06-09     0        0.12         1         1         4         3
 1      2016-06-09     1        0.11         1         1         4         3
 2      2016-06-09     2        0.11         1         1         4         3
 3      2016-06-09     3        0.12         1         1         4         3
 4      2016-06-09     4        0.12         1         1         4         3
 ...           ...   ...         ...       ...       ...       ...       ...
 30235  2019-11-20    19        0.51         1         1         4         3
 30236  2019-11-20    20        1.03         1         1         4         3
 30237  2019-11-20    21        2.10         1         1         4         3
 30238  2019-11-20    22        0.63         1         1         4         3
 30239  2019-11-20    23        0.21         1         1         4         3
 
 [30240 rows x 7 columns],
              date  hour  energy_kWh  kmeans_3 

In [ ]:
BATCH_SIZE = 32
NUM_CLIENTS = len(data_frames)

# Define a custom PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_loaders(data_frames, batch_size):
    cluster_loaders = []
    val_loaders = []
    test_loaders = []

    for data_frame in data_frames:
        # Split the current DataFrame into training, validation, and testing sets
        train_df, val_test_df = train_test_split(data_frame, test_size=0.4, random_state=42)
        val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

        # Convert cluster data to PyTorch tensors
        cluster_X_train = torch.tensor(train_df['hour'].values, dtype=torch.float32)
        cluster_y_train = torch.tensor(train_df['energy_kWh'].values, dtype=torch.float32).view(-1, 1)

        cluster_X_val = torch.tensor(val_df['hour'].values, dtype=torch.float32)
        cluster_y_val = torch.tensor(val_df['energy_kWh'].values, dtype=torch.float32).view(-1, 1)

        # Create DataLoader for the current dataset
        train_dataset = CustomDataset(cluster_X_train, cluster_y_train)
        cluster_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        cluster_loaders.append(cluster_loader)

        val_dataset = CustomDataset(cluster_X_val, cluster_y_val)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
        val_loaders.append(val_loader)

        # Convert testing data to tensors
        X_test = torch.tensor(test_df['hour'].values, dtype=torch.float32)
        y_test = torch.tensor(test_df['energy_kWh'].values, dtype=torch.float32).view(-1, 1)

        # Create DataLoader for testing data
        test_dataset = CustomDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        test_loaders.append(test_loader)

    return cluster_loaders, val_loaders, test_loaders


# Call the function to create loaders
trainloaders, valloaders, test_loader = create_loaders(data_frames, BATCH_SIZE)

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Take the last time step's output
        return output

In [ ]:
def train(model, dataloader, epochs: int, verbose=False):
    # Rest of the training function
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        running_loss = 0.0

        for inputs, target in dataloader:
            # Forward pass
            outputs = model(inputs.view(-1, 1).unsqueeze(1))

            # Compute the loss
            loss = criterion(outputs, target)

           # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

    print("Training complete")

def test(network, dataloader):
    criterion = nn.MSELoss()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = network(inputs.view(-1, 1).unsqueeze(1))
            loss = criterion(outputs, targets)
            total_loss += loss.item() * len(inputs)
            total_samples += len(inputs)

    loss = total_loss / total_samples

    return loss

In [ ]:
 # Create a ConcatDataset from the list of trainloaders
combined_dataset = ConcatDataset([dataset for dataset in trainloaders])
combined_val_dataset = ConcatDataset([dataset for dataset in valloaders])

# Define batch size and other DataLoader parameters
batch_size = 64
shuffle = True
num_workers = 4

# Create a DataLoader using the combined dataset
combined_trainloader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
combined_valloader = DataLoader(combined_val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)


trainloader = trainloaders[0]
valloader = valloaders[0]

input_size = 1
hidden_size = 64
output_size = 1

net = LSTMModel(input_size, hidden_size, output_size).to(DEVICE)

for epoch in range(2):
    train(net, trainloaders[epoch], 10)
    loss = test(net, valloaders[epoch])
    print(f"Epoch {epoch+1}: validation loss {loss}")

loss, accuracy = test(net, test_loader)
print(f"Final test set performance:\n\tloss")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: ignored

## Federated Learning

In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=50)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"loss": float(loss)}

In [ ]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    input_size = 1
    hidden_size = 64
    output_size = 1

    net = LSTMModel(input_size, hidden_size, output_size).to(DEVICE)


    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=1,  # Never sample less than 10 clients for training
    min_evaluate_clients=1,  # Never sample less than 5 clients for evaluation
    min_available_clients=1,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
client_resources = {'num_cpus': 2.0, 'num_gpus': 0.0}
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-10-15 11:26:18,336 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-10-15 11:26:25,356	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-10-15 11:26:29,422 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3907311206.0, 'memory': 7814622414.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3907311206.0, 'memory': 7814622414.0}
INFO flwr 2023-10-15 11:26:29,438 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 2.0, 'num_gpus': 0.0}
INFO:flwr:Flower VCE: Resources for each Virtual Client: {'num_cpus': 2.0, 'num_gpus': 0.0}
INFO flwr 2023-10-15 11:26:29,505 | app.py:270 | Flower VCE: Creat

(DefaultActor pid=48799) Epoch [1/50], Loss: 1.3391
(DefaultActor pid=48799) Epoch [2/50], Loss: 0.7459
(DefaultActor pid=48799) Epoch [3/50], Loss: 0.3633
(DefaultActor pid=48799) Epoch [4/50], Loss: 0.1245
(DefaultActor pid=48799) Epoch [5/50], Loss: 0.6112
(DefaultActor pid=48799) Epoch [6/50], Loss: 0.3319
(DefaultActor pid=48799) Epoch [7/50], Loss: 0.1297
(DefaultActor pid=48799) Epoch [8/50], Loss: 0.8418
(DefaultActor pid=48799) Epoch [9/50], Loss: 0.5749
(DefaultActor pid=48799) Epoch [10/50], Loss: 0.3011
(DefaultActor pid=48799) Epoch [11/50], Loss: 0.5152
(DefaultActor pid=48799) Epoch [12/50], Loss: 0.2681
(DefaultActor pid=48799) Epoch [13/50], Loss: 0.2772
(DefaultActor pid=48799) Epoch [14/50], Loss: 0.1346
(DefaultActor pid=48799) Epoch [15/50], Loss: 0.2499
(DefaultActor pid=48799) Epoch [16/50], Loss: 0.6934
(DefaultActor pid=48799) Epoch [17/50], Loss: 0.3510
(DefaultActor pid=48799) Epoch [18/50], Loss: 0.1960
(DefaultActor pid=48799) Epoch [19/50], Loss: 0.3769
(D

DEBUG flwr 2023-10-15 11:28:53,731 | server.py:236 | fit_round 1 received 2 results and 0 failures
DEBUG:flwr:fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-10-15 11:28:53,741 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-10-15 11:28:53,745 | server.py:173 | evaluate_round 1: strategy sampled 1 clients (out of 2)
DEBUG:flwr:evaluate_round 1: strategy sampled 1 clients (out of 2)
DEBUG flwr 2023-10-15 11:28:53,926 | server.py:187 | evaluate_round 1 received 1 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 1 results and 0 failures
WARNING flwr 2023-10-15 11:28:53,930 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-10-15 11:28:53,932 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG:flwr:fit_round 2: strategy sampled 2 clients (out of 2)


(DefaultActor pid=48799) Epoch [50/50], Loss: 0.0282
(DefaultActor pid=48799) Training complete
(DefaultActor pid=48799) Epoch [1/50], Loss: 0.2387
(DefaultActor pid=48799) Epoch [2/50], Loss: 1.2876
(DefaultActor pid=48799) Epoch [3/50], Loss: 0.1470
(DefaultActor pid=48799) Epoch [4/50], Loss: 0.0832
(DefaultActor pid=48799) Epoch [5/50], Loss: 0.2675
(DefaultActor pid=48799) Epoch [6/50], Loss: 0.5522
(DefaultActor pid=48799) Epoch [7/50], Loss: 1.5548
(DefaultActor pid=48799) Epoch [8/50], Loss: 0.4497
(DefaultActor pid=48799) Epoch [9/50], Loss: 0.3702
(DefaultActor pid=48799) Epoch [10/50], Loss: 0.3030
(DefaultActor pid=48799) Epoch [11/50], Loss: 0.2587
(DefaultActor pid=48799) Epoch [12/50], Loss: 0.3109
(DefaultActor pid=48799) Epoch [13/50], Loss: 0.3296
(DefaultActor pid=48799) Epoch [14/50], Loss: 0.5404
(DefaultActor pid=48799) Epoch [15/50], Loss: 0.5850
(DefaultActor pid=48799) Epoch [16/50], Loss: 0.2041
(DefaultActor pid=48799) Epoch [17/50], Loss: 0.3573
(DefaultActo

DEBUG flwr 2023-10-15 11:31:08,815 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG:flwr:fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-10-15 11:31:08,826 | server.py:173 | evaluate_round 2: strategy sampled 1 clients (out of 2)
DEBUG:flwr:evaluate_round 2: strategy sampled 1 clients (out of 2)
DEBUG flwr 2023-10-15 11:31:09,073 | server.py:187 | evaluate_round 2 received 1 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 1 results and 0 failures
DEBUG flwr 2023-10-15 11:31:09,078 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG:flwr:fit_round 3: strategy sampled 2 clients (out of 2)


...

(DefaultActor pid=48799) Epoch [50/50], Loss: 0.0199
(DefaultActor pid=48799) Training complete
(DefaultActor pid=48799) Epoch [1/50], Loss: 0.0322
(DefaultActor pid=48799) Epoch [2/50], Loss: 0.0366
(DefaultActor pid=48799) Epoch [3/50], Loss: 0.3065
(DefaultActor pid=48799) Epoch [4/50], Loss: 0.2271
(DefaultActor pid=48799) Epoch [5/50], Loss: 0.1108
(DefaultActor pid=48799) Epoch [6/50], Loss: 0.5106
(DefaultActor pid=48799) Epoch [7/50], Loss: 0.1299
(DefaultActor pid=48799) Epoch [8/50], Loss: 0.2133
(DefaultActor pid=48799) Epoch [9/50], Loss: 0.0564
(DefaultActor pid=48799) Epoch [10/50], Loss: 0.1388
(DefaultActor pid=48799) Epoch [11/50], Loss: 0.0803
(DefaultActor pid=48799) Epoch [12/50], Loss: 0.3486
(DefaultActor pid=48799) Epoch [13/50], Loss: 0.3313
(DefaultActor pid=48799) Epoch [14/50], Loss: 0.3238
(DefaultActor pid=48799) Epoch [15/50], Loss: 0.0318
(DefaultActor pid=48799) Epoch [16/50], Loss: 0.2790
(DefaultActor pid=48799) Epoch [17/50], Loss: 0.3521
(DefaultActo

DEBUG flwr 2023-10-15 11:49:40,783 | server.py:236 | fit_round 10 received 2 results and 0 failures
DEBUG:flwr:fit_round 10 received 2 results and 0 failures
DEBUG flwr 2023-10-15 11:49:40,793 | server.py:173 | evaluate_round 10: strategy sampled 1 clients (out of 2)
DEBUG:flwr:evaluate_round 10: strategy sampled 1 clients (out of 2)


(DefaultActor pid=48799) Epoch [50/50], Loss: 0.3408
(DefaultActor pid=48799) Training complete


DEBUG flwr 2023-10-15 11:49:40,991 | server.py:187 | evaluate_round 10 received 1 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 1 results and 0 failures
INFO flwr 2023-10-15 11:49:40,994 | server.py:153 | FL finished in 1382.1651291430007
INFO:flwr:FL finished in 1382.1651291430007
INFO flwr 2023-10-15 11:49:40,997 | app.py:225 | app_fit: losses_distributed [(1, 0.8889704174901161), (2, 0.7337824119132093), (3, 0.7507855379860158), (4, 0.7398609314648468), (5, 0.8063887364736099), (6, 0.9042268398142976), (7, 0.16500947596850218), (8, 0.165030868852107), (9, 0.17740400829327801), (10, 0.19436079070523934)]
INFO:flwr:app_fit: losses_distributed [(1, 0.8889704174901161), (2, 0.7337824119132093), (3, 0.7507855379860158), (4, 0.7398609314648468), (5, 0.8063887364736099), (6, 0.9042268398142976), (7, 0.16500947596850218), (8, 0.165030868852107), (9, 0.17740400829327801), (10, 0.19436079070523934)]
INFO flwr 2023-10-15 11:49:41,000 | app.py:226 | app_fit: metrics_distributed_f

History (loss, distributed):
	round 1: 0.8889704174901161
	round 2: 0.7337824119132093
	round 3: 0.7507855379860158
	round 4: 0.7398609314648468
	round 5: 0.8063887364736099
	round 6: 0.9042268398142976
	round 7: 0.16500947596850218
	round 8: 0.165030868852107
	round 9: 0.17740400829327801
	round 10: 0.19436079070523934